## Mount to MongoDB
MongoDB is a useful NoSQL database for storing large amounts of JSON. We can run this on a server, or locally if we have enough drive space. 

Compared with scanning the data files, MongoDB can allow us to index the data, this way we can pick which specific data we want to look at more easily. 

In [1]:
import pymongo
import json
import gzip
import os
import datetime

In [2]:
# initialise client class for interacting with db
client = pymongo.MongoClient()
# create or open database
db = client['unpaywall']
# create or open collection (table) within database
collection = db['snapshot']

In [3]:
# check
# collection.find_one()

In [4]:
filepath = r'C:\Users\aday\OneDrive - SAGE Publishing\DATA\Unpaywall\unpaywall_snapshot_2021-02-18T160139.jsonl.gz'

## Upserting Vs Inserting
- Inserting is where we simply put a new record into our database. It is the fastest way to CREATE our database from scratch in 1 go. You can also use batch processes to do this faster (but below we only insert one at a time). 
- If we want to add to our database (i.e. create in multiple gos), or update it with a new snapshot, then we can upsert instead. This way, we check to see if our document already exists in the data. 
  - If not, then we insert it.
  - If so, and our document has been updated, then we update the document in our database.
  - If both documents are identical, we skip the update. 

Upserting avoids unecessarily writing the the database, which can be slow. __Note__ that to upsert data, you need to have an index on the doi field. This is done in the cell below the next one. Be aware that, the larger your collection, the slower creating the index will be. 

In [5]:
import datetime
import gzip
batch_size = 1000
batch = []
with gzip.open(filepath,'rb') as f:
    
    for i,line in enumerate(f):
        
        
        # INSERT DATA
#         if i<20000000:
#             continue
#         else:
#             try:
#                 collection.insert_one(json.loads(line))
#             except Exception as e:
#                 print('Error: ', e)
#                 print('Bad line: ', line)
#                 print()
#         if i%1000000==0:
#             print(i, 'records done')


        ## UPSERT DATA
        line = json.loads(line)
        doi = line['doi']
        
        
        batch.append(pymongo.UpdateOne({'doi':doi}, {'$set':line}, upsert = True))
        try:
            if i%batch_size==0:
                collection.bulk_write(batch)
                batch = []
                if i%(1000*batch_size)==0:
                    print(datetime.datetime.now(),i, 'records written to Mongo')
        except Exception as e:
            print('ERROR', e)
            print('Batch write failed! Try items individually')
            for item in batch:
                try:
                    collection.bulk_write([item])
                except Exception as e:
                    print('Error: ', e)
                    print('ITEM:', item)
                batch = []
    # Final batch
    if len(batch)>0:
        try:
            collection.bulk_write(batch)
        except:
            print('Batch write failed! Try items individually')
            for item in batch:
                try:
                    collection.write(item)
                except Exception as e:
                    print('Error: ', e)
                    print('ITEM:', item)
    else:
        print('No records to write. Skipping')

2021-02-22 13:42:14.432223 20000000 records written to Mongo
2021-02-22 13:53:17.333130 21000000 records written to Mongo
2021-02-22 14:05:10.295292 22000000 records written to Mongo
2021-02-22 14:16:18.747638 23000000 records written to Mongo
2021-02-22 14:26:53.700182 24000000 records written to Mongo
2021-02-22 14:38:20.724211 25000000 records written to Mongo
2021-02-22 14:51:38.389285 26000000 records written to Mongo
2021-02-22 15:08:15.265702 27000000 records written to Mongo
2021-02-22 15:25:05.560059 28000000 records written to Mongo
2021-02-22 15:40:38.005620 29000000 records written to Mongo
2021-02-22 15:54:36.483126 30000000 records written to Mongo
2021-02-22 16:06:17.736212 31000000 records written to Mongo
2021-02-22 16:16:32.765132 32000000 records written to Mongo
2021-02-22 16:26:43.826653 33000000 records written to Mongo
2021-02-22 16:37:00.626381 34000000 records written to Mongo
2021-02-22 16:47:33.917943 35000000 records written to Mongo
2021-02-22 16:57:46.5819

## Create Indexes

In [6]:
# create index (or simply confirm it's there already)
collection.create_index([( 'doi', pymongo.ASCENDING )])

In [ ]:
collection.create_index([( 'year', pymongo.ASCENDING )])

In [ ]:
collection.create_index([( 'publisher', pymongo.ASCENDING )])

## Check
How quickly can we retrieve an item by its doi? Pick any DOI you like!

In [7]:
%%time
collection.find_one({'doi': '10.1080/21645515.2017.1330236'})

Wall time: 5.01 ms


{'_id': ObjectId('5fbc111ce91f8c83cf0fc86c'),
 'doi': '10.1080/21645515.2017.1330236',
 'year': 2017,
 'genre': 'journal-article',
 'is_oa': True,
 'title': 'Advancements in DNA vaccine vectors, non-mechanical delivery methods, and molecular adjuvants to increase immunogenicity',
 'doi_url': 'https://doi.org/10.1080/21645515.2017.1330236',
 'updated': '2021-01-17T06:00:02.039312',
 'oa_status': 'hybrid',
 'publisher': 'Informa UK Limited',
 'z_authors': [{'given': 'John J.',
   'family': 'Suschak',
   'sequence': 'first',
   'affiliation': [{'name': 'U.S. Army Medical Research Institute of Infectious Diseases, Fort Detrick, MD, USA'}]},
  {'given': 'James A.',
   'family': 'Williams',
   'sequence': 'additional',
   'affiliation': [{'name': 'Nature Technology Corporation, Lincoln, NE, USA'}]},
  {'given': 'Connie S.',
   'family': 'Schmaljohn',
   'sequence': 'additional',
   'affiliation': [{'name': 'U.S. Army Medical Research Institute of Infectious Diseases, Fort Detrick, MD, USA'}]